In [1]:
from analytics_cloud_core import Clients, ClientType
import pandas as pd
import numpy as np
from datetime import date
from pygsuite import Spreadsheet
from google.cloud import bigquery

In [2]:
def read_google_spreadsheet(sheet_id, sheet_name):

    # authenticate to the pygsuite package
    Clients.get_client(ClientType.PYGSUITE)
    sheet = Spreadsheet(id = sheet_id)
    df = sheet.worksheets[sheet_name].dataframe

    return df

sheet_id = '1GxuvblcWKHmYaAVHLfk0QGLHm0eHDIkA1CGbI_4EsA4'
web_sheet_name = 1
app_sheet_name = 2

web = read_google_spreadsheet(sheet_id, web_sheet_name)
app = read_google_spreadsheet(sheet_id, app_sheet_name)

web = web[web['Page']!=""]
app = app[app['Page']!=""]
gsheet = pd.concat([web,app])

In [3]:
client = bigquery.Client(project = 'wf-gcp-us-ae-sf-prod')

query = f"""
select * from `wf-gcp-us-ae-sf-prod.foundation_quality.tbl_page_type_budget`
"""
query_job = client.query(query)  # Make an API request.
past_slo = query_job.to_dataframe()

In [4]:
client = bigquery.Client(project = 'wf-gcp-us-ae-sf-prod')

query = f"""
select distinct pt.PageGroupName, 
  slo.PageID
from `wf-gcp-us-ae-sf-prod.foundation_quality.tbl_page_type_budget` slo
left join `wf-gcp-us-ae-sf-prod.performance.tbl_page_type_grouping` pt
on slo.pageid = pt.pageid
where startdate = '2022-01-01T00:00:00' and pagegroupname is not null
"""
query_job = client.query(query)  # Make an API request.
pageid = query_job.to_dataframe()

In [5]:
new_slo = gsheet.merge(pageid, how = 'left', left_on = 'Page', right_on = 'PageGroupName')[['PageGroupName','PageID','PlatformID','Percentile','SLO (New)']]
new_slo['Percentile'] = new_slo['Percentile'].str[-2:]
new_slo['SLO (New)'] = new_slo['SLO (New)'].astype(int)
new_slo['SpeedIndexBudget75th'] = new_slo[['PageID','PlatformID','Percentile','SLO (New)']].apply(
    lambda x: x['SLO (New)'] if x['Percentile'] == '75' else None, axis = 1)
new_slo['SpeedIndexBudget95th'] = new_slo[['PageID','PlatformID','Percentile','SLO (New)']].apply(
    lambda x: x['SLO (New)'] if x['Percentile'] == '95' else None, axis = 1)
new_slo = new_slo.drop(columns = ('Percentile'))
new_slo = new_slo.groupby(['PageGroupName','PageID','PlatformID']).agg(
    {'SpeedIndexBudget75th':'max','SpeedIndexBudget95th':'max'})
new_slo = new_slo.reset_index()

In [6]:
new_slo['StartDate'] = date.today()
new_slo['SpeedIndexBudget'] = np.nan
new_slo['SpeedIndexGoal'] = np.nan
new_slo['TotalPageLoadTimeBudget'] = np.nan
new_slo['TotalPageLoadTimeGoal'] = np.nan
new_slo['HTMLSizeBudget'] = np.nan
new_slo['HTMLSizeGoal'] = np.nan
new_slo['SpeedIndexBudget99th'] = np.nan 
new_slo['SpeedIndexGoal99th'] = np.nan
new_slo['TotalPageLoadTimeBudget99th']=np.nan
new_slo['TotalPageLoadTimeGoal99th'] = np.nan
new_slo['SpeedIndexGoal75th']=np.nan
new_slo['SpeedIndexGoal95th']=np.nan
new_slo['ingest_timestamp']= np.nan
new_slo['event_date']=np.nan
new_slo = new_slo.drop(columns = 'PageGroupName')
#new_slo.dtypes

In [7]:
#pd.set_option('display.max_rows', 5000)
update_slo_table = pd.concat([new_slo, past_slo])
update_slo_table = update_slo_table[['PageID', 'PlatformID', 'StartDate', 'SpeedIndexBudget',
       'SpeedIndexGoal', 'TotalPageLoadTimeBudget',
       'TotalPageLoadTimeGoal', 'HTMLSizeBudget', 'HTMLSizeGoal',
       'SpeedIndexBudget99th', 'SpeedIndexGoal99th',
       'TotalPageLoadTimeBudget99th', 'TotalPageLoadTimeGoal99th',
       'SpeedIndexBudget75th', 'SpeedIndexGoal75th',
       'SpeedIndexBudget95th', 'SpeedIndexGoal95th', 'ingest_timestamp',
       'event_date']]

update_slo_table['PlatformID']= update_slo_table['PlatformID'].astype(int)
update_slo_table['StartDate'] = pd.to_datetime(update_slo_table['StartDate'])
update_slo_table['ingest_timestamp'] = pd.to_datetime(update_slo_table['ingest_timestamp'])
update_slo_table['event_date'] = pd.to_datetime(update_slo_table['event_date'])
#update_slo_table.dtypes

In [8]:
#new_slo
update_slo_table

,PageID,PlatformID,StartDate,SpeedIndexBudget,SpeedIndexGoal,TotalPageLoadTimeBudget,TotalPageLoadTimeGoal,HTMLSizeBudget,HTMLSizeGoal,SpeedIndexBudget99th,SpeedIndexGoal99th,TotalPageLoadTimeBudget99th,TotalPageLoadTimeGoal99th,SpeedIndexBudget75th,SpeedIndexGoal75th,SpeedIndexBudget95th,SpeedIndexGoal95th,ingest_timestamp,event_date
0,125,1,2023-03-20 00:00:00.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,1.0,NaN,NaT,NaT
1,125,2,2023-03-20 00:00:00.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,3.0,NaN,NaT,NaT
2,125,3,2023-03-20 00:00:00.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,NaN,5.0,NaN,NaT,NaT
3,125,4,2023-03-20 00:00:00.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,NaN,7.0,NaN,NaT,NaT
4,114,1,2023-03-20 00:00:00.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,0.0,NaN,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4738,689,2,2019-03-13 18:39:08.537,1196.0,1177.0,2397.0,2347.0,261.0,163.0,10857.0,9209.0,25938.0,24837.0,2440.0,1856.0,5600.0,4265.0,2022-04-05 11:32:06.347644+00:00,2022-04-05
4739,475,2,2019-03-13 18:39:08.537,1192.0,1159.0,2700.0,2584.0,271.0,171.0,9246.0,7676.0,25532.0,18663.0,1220.0,929.0,2800.0,2134.0,2022-04-05 11:32:06.347644+00:00,2022-04-05
4740,711,2,2019-03-13 18:39:08.537,1389.0,1153.0,2506.0,2116.0,147.0,79.0,14228.0,14228.0,21908.0,21908.0,4830.0,3679.0,6390.0,4871.0,2022-04-05 11:32:06.347644+00:00,2022-04-05
4741,1036,2,2019-03-13 18:39:08.537,598.0,566.0,1203.0,1153.0,165.0,94.0,2998.0,2990.0,10733.0,9622.0,850.0,643.0,1530.0,1162.0,2022-04-05 11:32:06.347644+00:00,2022-04-05


In [9]:
client = bigquery.Client(project = 'wf-gcp-us-ae-sf-prod')

# run update for update table
job_config = bigquery.LoadJobConfig(schema=[
    bigquery.SchemaField(name="PageID", field_type="INTEGER"),
    bigquery.SchemaField(name="PlatformID", field_type="INTEGER"),
    bigquery.SchemaField(name="StartDate", field_type="DATETIME"),
    bigquery.SchemaField(name="SpeedIndexBudget", field_type="INTEGER"),
    bigquery.SchemaField(name="SpeedIndexGoal", field_type="INTEGER"),
    bigquery.SchemaField(name="TotalPageLoadTimeBudget", field_type="INTEGER"),
    bigquery.SchemaField(name="TotalPageLoadTimeGoal", field_type="INTEGER"),
    bigquery.SchemaField(name="HTMLSizeBudget", field_type="INTEGER"),
    bigquery.SchemaField(name="HTMLSizeGoal", field_type="INTEGER"),
    bigquery.SchemaField(name="SpeedIndexBudget99th", field_type="INTEGER"),
    bigquery.SchemaField(name="SpeedIndexGoal99th", field_type="INTEGER"),
    bigquery.SchemaField(name="TotalPageLoadTimeBudget99th", field_type="INTEGER"),
    bigquery.SchemaField(name="TotalPageLoadTimeGoal99th", field_type="INTEGER"),
    bigquery.SchemaField(name="SpeedIndexBudget75th", field_type="INTEGER"),
    bigquery.SchemaField(name="SpeedIndexGoal75th", field_type="INTEGER"),
    bigquery.SchemaField(name="SpeedIndexBudget95th", field_type="INTEGER"),
    bigquery.SchemaField(name="SpeedIndexGoal95th", field_type="INTEGER"),
    bigquery.SchemaField(name="ingest_timestamp", field_type="TIMESTAMP"),
    bigquery.SchemaField(name="event_date", field_type="DATE")], write_disposition="WRITE_APPEND")
update_t = client.load_table_from_dataframe(update_slo_table,
                                            'wf-gcp-us-ae-sf-prod.junk.si_slo_update_test',
                                            job_config=job_config)
update_t.result()

LoadJob<project=wf-gcp-us-ae-sf-prod, location=US, id=2a4d00bf-29d0-43ad-a2ae-e32cfc55317b>